In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [2]:
def softmax(x):
    exp = np.exp(x)
    return exp / np.sum(exp)

In [3]:
def rescorla_wagner(q_val, epsilon, reward):
    return q_val + epsilon * (reward - q_val)

In [4]:
def reward(r_t, rho_rew, rho_pun):
    if r_t > 0: return rho_rew
    if r_t < 0: return -rho_pun
    return 0

In [5]:
def log_likelihood(cues, actions, rewards, epsilon, rho_rew, rho_pun):
    n_stimuli = len(set(cues))
    n_actions = len(set(actions))

    q_vals = np.zeros((n_stimuli, n_actions))

    log_likelihood = 0

    for t, a_t in enumerate(actions):
        s_t = cues[t] - 1
        r_t = reward(rewards[t], rho_rew, rho_pun)

        probs = softmax(q_vals[s_t])
        log_likelihood += np.log(probs[a_t])

        # Update the Q-values using Rescorla-Wagner
        q_vals[s_t, a_t] = rescorla_wagner(
            q_val = q_vals[s_t, a_t],
            epsilon = epsilon,
            reward = r_t
        )

    return log_likelihood

In [6]:
data = pd.read_csv("gen_data.csv")

In [7]:
def loss(params, cues, actions, rewards):
    epsilon, rho_rew, rho_pun = params
    return -log_likelihood(cues, actions, rewards, epsilon, rho_rew, rho_pun)

In [8]:
epsilon_bounds = (0.0000001, 0.99999)
beta_bounds = (0.0001, 9.9999)

In [9]:
min_loss = []

for subject_id in data.ID.unique():
    subject = data[ data.ID == subject_id ]

    cues = subject.cue.tolist()
    actions = subject.pressed.tolist()
    rewards = subject.outcome.tolist()

    res = minimize(
        fun = loss,
        x0 = [0.5, 5, 5],
        bounds = [epsilon_bounds, beta_bounds, beta_bounds],
        args = (cues, actions, rewards),
        method = "Nelder-Mead"
    )

    min_loss.append(res.fun)

In [10]:
np.sum(min_loss)

2862.9286280091674